In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
from sklearn.utils import shuffle

In [3]:
trainX = pd.read_csv('train_MLWARE1.csv')
test = pd.read_csv('test_MLWARE1.csv')

In [4]:
train = shuffle(trainX, random_state=25)

In [5]:
train = train.reset_index()

In [6]:
train.head(10)

,index,ID,tweet,label
0,21590,T000473948,@beasangel you're you're new to the internet a...,sarcastic
1,48598,T000500956,b'If congress wanted to make it simpler they w...,sarcastic
2,44788,T000497146,"#sarcasm sorry thought you'd catch it""",sarcastic
3,5541,T000457899,b'the changes made to the sat totally undermin...,sarcastic
4,58456,T000510814,b'just dance was just announced on i love the ...,non-sarcastic
5,18773,T000471131,really love it when niggas sweat them #not me',sarcastic
6,61927,T000514285,drinks delicious steak salad best company love...,non-sarcastic
7,6713,T000459071,b'Los de en Sambueza de verdad Y con Boy mezcl...,sarcastic
8,52234,T000504592,b'rt turning left onto steeles ',non-sarcastic
9,8370,T000460728,b'Wow glad my alarm is going off in hours $$SA...,sarcastic


In [7]:
test.head(2)

,ID,tweet
0,T000543656,and baby showers the new weddings lmao'
1,T000543657,in only white people can be racist if a white ...


In [8]:
tweets = pd.DataFrame(train.tweet)
tweets.tweet[0]

"@beasangel you're you're new to the internet aren't you"

In [9]:
len(tweets)

91298

In [10]:
train.shape

(91298, 4)

# COUNTING word frequency

In [11]:
total_counts = Counter()
saracstic_counts = Counter()
non_sarcastic_counts = Counter()

for word in train.tweet.values:
    total_counts.update(word.split(' '))
    
print(len(total_counts))

57562


In [12]:
total_counts.most_common(10)

[('to', 31931),
 ('the', 29937),
 ('a', 21607),
 ('and', 14796),
 ('for', 14677),
 ('you', 14186),
 ('my', 13507),
 ('of', 13479),
 ('is', 13208),
 ('in', 12667)]

In [13]:
vocab = sorted(total_counts, key= total_counts.get , reverse = True)[:8000]
print(vocab[-1], total_counts[vocab[-1]])

WHEN 10


In [14]:
word2idx = {word :i for i, word in enumerate(vocab)}

In [15]:
word2idx

{'mostly': 3511,
 '#Seriously': 995,
 'although': 5180,
 'application': 6427,
 '"i': 2671,
 'Alex': 6633,
 'among': 3708,
 '#gif': 1895,
 "b'Maybe": 4158,
 'friends"': 7962,
 "It's": 323,
 'base': 3745,
 'telling': 774,
 'His': 3070,
 'teach': 1690,
 'ON': 2790,
 'boxing': 5265,
 'Daniel': 6495,
 'lee': 5060,
 '#be': 5049,
 'salad': 4629,
 'eat': 729,
 'forever': 1038,
 'picnic': 7012,
 "b'An": 5528,
 'heritage': 7138,
 'open': 491,
 'mess': 2317,
 'WAS': 5203,
 'exists': 4955,
 "b'join": 5612,
 'for"': 4480,
 'potter': 3638,
 'innovation': 7287,
 'nights': 909,
 'event': 662,
 'restaurant': 2659,
 'truly': 884,
 'mug': 7139,
 'jesus': 4257,
 "b'Let": 4856,
 'london': 779,
 'basically': 2449,
 'shopping': 1157,
 "they'll": 4707,
 'striker': 6718,
 'Jack': 6844,
 'b"omg': 2488,
 'credit': 1787,
 'cough': 5885,
 'yolo': 6428,
 'invaded': 7851,
 'months': 554,
 'thanx': 6429,
 'interview': 1110,
 'killed': 1967,
 'English': 1699,
 'never': 156,
 'Yes': 1151,
 'ban': 2233,
 "here's": 1724,

In [16]:
print(len(vocab))

8000


In [17]:
def text_to_vector(text):
    word2vec = np.zeros(len(vocab) , dtype = np.int_)
    
    for word in text.split(' '):
        idx = word2idx.get(word , None)
        if idx is None:
            continue
        else:
            word2vec[idx] = 1
            
    return np.array(word2vec)

In [18]:
text_to_vector('the motion to the sales are in  a motion')

array([1, 1, 1, ..., 0, 0, 0])

In [19]:
word_vectors = np.zeros( (len(tweets),len(vocab)), dtype = np.int_)

In [20]:
word_vectors.shape

(91298, 8000)

In [21]:
for i, (_, word) in enumerate(tweets.iterrows()):
    word_vectors[i] = text_to_vector(word[0])

In [22]:
word_vectors[:5, :20]

array([[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0]])

In [23]:
labels = train.label

In [24]:
Y = (labels=='sarcastic').astype(np.int_)

In [25]:
y = to_categorical(Y.values, 2)

In [26]:
y[0:10]

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.]])

In [27]:
test_tweets = pd.DataFrame(test.tweet)
test_word2vectors = np.zeros( (len(test_tweets),len(vocab)), dtype = np.int_)
for i, (_, word) in enumerate(test_tweets.iterrows()):
    test_word2vectors[i] = text_to_vector(word[0])

In [28]:
#from sklearn.model_selection import train_test_split

In [29]:
#records = len(labels)
#shuffle = np.arange(records)
#np.random.shuffle(shuffle)

In [30]:
#print(len(shuffle),len(word_vectors))

In [31]:
#shuffle

In [32]:
#trainY = y[shuffle,:]

In [33]:
# X_train, X_test, y_train, y_test = train_test_split(word_vectors, y, test_size=0.10, random_state=42)

In [34]:
#trainY

## BUILD MODEL

In [35]:
def build_model():
    tf.reset_default_graph()
    
    # Inputs
    net = tflearn.input_data([None, 8000])

    # Hidden layer(s)
    net = tflearn.fully_connected(net, 200, activation='ReLU')
    net = tflearn.fully_connected(net, 25, activation='ReLU')

    # Output layer
    net = tflearn.fully_connected(net, 2, activation='softmax')
    net = tflearn.regression(net, optimizer='sgd', 
                             learning_rate=0.1, 
                             loss='categorical_crossentropy')
    
    model = tflearn.DNN(net)
    return model

In [ ]:
model = build_model()

In [ ]:
model.fit(word_vectors, y, validation_set=0.1, show_metric=True, batch_size=128, n_epoch=45)

Training Step: 26431  | total loss: 0.00398 | time: 9.484s
| SGD | epoch: 042 | loss: 0.00398 - acc: 0.9995 -- iter: 13952/82168


In [41]:
print('done')

done


In [42]:
model

In [43]:
predictions = model.predict(test_word2vectors)

In [44]:
predictions

[[0.9947689771652222, 0.005231111776083708],
 [6.443104211939499e-06, 0.9999935626983643],
 [2.2035755274340416e-22, 1.0],
 [0.0003371028578840196, 0.9996628761291504],
 [0.977256178855896, 0.022743787616491318],
 [0.9997507929801941, 0.0002492301573511213],
 [0.9999817609786987, 1.8218330296804197e-05],
 [0.9989769458770752, 0.0010231005726382136],
 [3.419073254917748e-05, 0.9999657869338989],
 [0.999997615814209, 2.425954562568222e-06],
 [0.2519241273403168, 0.7480759024620056],
 [0.643126368522644, 0.35687360167503357],
 [0.9999291896820068, 7.083712262101471e-05],
 [1.2985088915229426e-06, 0.9999986886978149],
 [0.9999954700469971, 4.545658612187253e-06],
 [4.743486159419774e-11, 1.0],
 [0.9998960494995117, 0.00010395507706562057],
 [0.9873895645141602, 0.01261039450764656],
 [0.20674630999565125, 0.7932537198066711],
 [0.9999313354492188, 6.870926154078916e-05],
 [0.03159775584936142, 0.9684022068977356],
 [0.9999667406082153, 3.3241794881178066e-05],
 [0.9996892213821411, 0.00031

In [56]:
predictions[0][0]

0.9947689771652222

In [47]:
predictions_f = (predictions[:,0] >= 0.5).astype(np.int_)

SyntaxError: invalid syntax (<ipython-input-47-a2bc28e6185e>, line 1)